# Fit sensors
Example of how to convert from one sensor to another, not the best way but often it's enough.

In [40]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Load the CSVs
ref_df = pd.read_csv('data/reference.csv', sep = ';')
sens_df = pd.read_csv('data/sensor_data.csv', sep = ';')

# Convert to datetime
ref_df['date'] = pd.to_datetime(ref_df['date'], format='%H:%M:%S')
sens_df['date'] = pd.to_datetime(sens_df['date'], format='%H:%M:%S')

# Convert to floats, excel uses commas instead of dots
# ref_df['umol/(m^2*s)'] = ref_df['umol/(m^2*s)'].str.replace(',', '.').astype(float)
sens_df['Counts'] = sens_df['Counts'].str.replace(',', '.').astype(float)

# Round to minutes
ref_df['rounded_date'] = ref_df['date'].dt.round('min')
sens_df['rounded_date'] = sens_df['date'].dt.round('min')

# Resample to 5-minute intervals and calculate the mean
ref_df.set_index('date', inplace=True)
ref_df = ref_df.resample('5min').mean()
ref_df.reset_index(inplace=True)
ref_df['rounded_date'] = ref_df['date']

sens_df.set_index('date', inplace=True)
sens_df = sens_df.resample('5min').mean()
sens_df.reset_index(inplace=True)
sens_df['rounded_date'] = sens_df['date']

ref_df.head()

In [ ]:
sens_df.head()

In [ ]:
# Merge the two dataframes to create a 'lookup' table from the sensor to the reference
merged_df = pd.merge(sens_df, ref_df, on='rounded_date', how='left')
# Drop all rows with any NaN values
merged_df = merged_df.dropna()
merged_df

In [ ]:
# Scatterplot
fig = go.Figure()
fig.add_trace(go.Scatter(x=merged_df['Counts'], y=merged_df['umol/(m^2*s)'], mode='markers', name='Data'))
fig.update_layout(title='Counts vs umol/(m^2*s)',
                  xaxis_title='Counts',
                  yaxis_title='umol/(m^2*s)')

fig.show()

In [51]:
# Sort the DataFrame by 'Counts' to ensure x in increaing, required for the fits
merged_df = merged_df.sort_values(by='Counts')

# Looks liniear, fit a line through the data
x = merged_df['Counts']
y = merged_df['umol/(m^2*s)']
slope, intercept = np.polyfit(x, y, 1)

# Fit a thrird order line
coefficients = np.polyfit(x, y, 3)
polynomial = np.poly1d(coefficients)

In [ ]:
# Plot data with lines
x_smooth = np.linspace(x.min(), x.max(), 500)
y_lin = slope * x_smooth + intercept
y_3thord = polynomial(x_smooth)

# Create the plot
fig = go.Figure()
# Add scatter plot for original data
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data'))

# Add line plot for linear interpolation
fig.add_trace(go.Scatter(x=x_smooth, y=y_lin, mode='lines', name='Linear Interpolation'))

# Add line plot for spline interpolation
fig.add_trace(go.Scatter(x=x_smooth, y=y_3thord, mode='lines', name='3th order polynomial'))

# Update layout
fig.update_layout(title='Counts vs umol/(m^2*s) with Linear and Spline Interpolations',
                  xaxis_title='Counts',
                  yaxis_title='umol/(m^2*s)')

# Show the plot
fig.show()

In [ ]:
# Calculate the residuals and RMSE
poly_1d = np.poly1d([slope, intercept])
y_fit = poly_1d(x)
residual_lin = y - y_fit
rmse = np.sqrt(np.mean(residual_lin**2))
print("Liniear fit")
print(f"Function: {slope:.2f}x + {intercept:.2f}")
print(f"Root Mean Square Error (RMSE): {rmse} \n")

# Print the coefficients and RMSE
y_fit = polynomial(x)
residuals = y - y_fit
rmse = np.sqrt(np.mean(residuals**2))
print("3th order fit")
print(f"Polynomial coefficients: {coefficients}")
print(f"Root Mean Square Error (RMSE): {rmse}")

In [ ]:
# Residual plot
# If the residuals look randomly distributed around zero without any discernible patterns, it indicates a good fit.
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=residual_lin, mode='markers', name='Residuals liniear fit'))
fig.add_trace(go.Scatter(x=x, y=residuals, mode='markers', name='Residuals poly fit'))
# Add horizontal line at y=0
fig.add_trace(go.Scatter(x=[x.min(), x.max()], y=[0, 0], mode='lines', name='Zero Line', line=dict(color='red', dash='dash')))

# Update layout
fig.update_layout(title='Residual Plot',
                  xaxis_title='Counts',
                  yaxis_title='Residuals')

# Show the plot
fig.show()